精细地控制AI模型的输出,以获得最符合需求的结果

In [ ]:
system instruction

安全设置可以在 genai.GenerativeModel 构造函数中设置。它们也可以在每个请求中传递给 GenerativeModel.generate_content 或 ChatSession.send_message。

generation_config参数里：

1. candidate_count:
   这个参数决定模型会生成多少个候选回答。比如设为3,模型就会给出3个不同的回答供选择。**但是目前，该值只能设置为 1。如果未设置，则默认为 1。**

3. max_output_tokens:
   这限制了模型生成内容的最大长度。一个token大致相当于一个单词或标点符号。设置为100就是限制生成大约100个单词的内容。

2. stop_sequences:
   这是一组字符串,用来告诉模型在生成到这些字符串时停止。比如设置为["。","!"],模型生成内容时遇到句号或感叹号就会停止。



4. temperature:
   这控制输出的随机性。值从0到1,越接近1输出越有创意性和多样性,越接近0输出越确定和保守。

5. top_p:
   这也用来控制随机性。它决定模型只考虑累积概率达到这个值的词。比如设为0.9,模型只会从累积概率前90%的词中选择。

6. top_k:
   这限制了模型在每一步只考虑概率最高的前k个词。默认是40,意味着每次只从最可能的40个词中选择。

7. response_mime_type:
   这指定了输出的格式。可以是普通文本(text/plain)或JSON格式(application/json)。

8. response_schema:
   如果输出是JSON格式,这个参数可以指定JSON的具体结构。


    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 1,
        "max_output_tokens": 2048,
    }


In [67]:
import os

import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"], transport='rest')
model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827')

max_output_tokens參數

1. Token的定义

在大语言模型中,token是文本的基本单位。模型不是直接处理原始文本,而是将文本切分成一系列token进行处理，最后将token映射成数字。token可以是单词、子词或者单个字符,具体取决于模型使用的分词方法。
 - 例如，句子 "I love AI" 可能被分成 ["I", "love", "AI"] 这样的tokens。同时一个单词可能被分成多个tokens。比如 "understanding" 可能被分成 ["under", "standing"]。
 - 英文中,1个token大约对应0.75个单词或4个字符。中文中,1个token大约对应1.5个汉字。

2. 输入和输出Token
输入tokens (读取过程)

- 作用: 用于模型理解和处理输入的文本。限制了模型一次可以处理的输入文本长度。
- 过程: 输入文本被tokenize成一系列token,然后将token映射成数字。

输出tokens (生成过程)

- 作用: 控制模型生成的文本长度。限制了模型一次可以生成的文本长度
- 过程: 模型逐个生成token,直到达到停止条件或token数量上限。

输入token限制:
1. 1.5 Flash: 约104万tokens
2. 1.5 Pro: 约209万tokens
而现在的竞品Claude 3.5 Sonnet: 20万tokens

token和多模态：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240923152248.png)


1.5 Pro列出了多模态处理能力，2小时的视频，19小时的音频，60,000行代码，2,000页文本


3. max_output_tokens参数控制输出tokens：
max_output_tokens参数用于控制模型的输出长度。当生成的token数量达到这个限制时，模型会被强制停止生成，无论当前的句子或思路是否完整。

- 模型会在达到token数量上限时停止生成，可能导致输出不完整,句子被截断
- 有助于控制计算资源使用和响应时间
- 如果未设置,将默认为模型规格中指定的output_token_limit。


In [12]:



def get_history_question():
    questions = [
        {
            "question": "秦始皇陵中的兵马俑最初是什么颜色的？",
            "options": ["A. 灰色", "B. 彩色", "C. 黑色", "D. 白色"],
            "answer": "B"
        },
        {
            "question": "下列哪位不是中国古代四大发明家之一？",
            "options": ["A. 蔡伦", "B. 毕昇", "C. 张衡", "D. 李时珍"],
            "answer": "D"
        },
        {
            "question": "古埃及金字塔最初是用来做什么的？",
            "options": ["A. 皇家宫殿", "B. 天文观测站", "C. 法老的陵墓", "D. 宗教祭祀场所"],
            "answer": "C"
        },
        {
            "question": "古罗马斗兽场（罗马竞技场）最初的名字是什么？",
            "options": ["A. 凯旋门", "B. 弗拉维圆形剧场", "C. 万神殿", "D. 特洛伊竞技场"],
            "answer": "B"
        },
        {
            "question": "中国古代哪个朝代发明了火药？",
            "options": ["A. 唐朝", "B. 宋朝", "C. 元朝", "D. 明朝"],
            "answer": "A"
        }
    ]
    question = random.choice(questions)
    question_text = question["question"]
    for option in question["options"]:
        question_text += option + "\n"
    right_answer = question["answer"]
    return question_text, right_answer


for i in range(2):
    question_text, right_answer = get_history_question()
    print(question_text)
    response = model.generate_content(contents=question_text + "直接回答选项的字母", generation_config={
        "max_output_tokens": 1,
    })
    print(f"大模型的答案是{response.text},正确答案是{right_answer}")

古埃及金字塔最初是用来做什么的？A. 皇家宫殿
B. 天文观测站
C. 法老的陵墓
D. 宗教祭祀场所

大模型的答案是C,正确答案是C
中国古代哪个朝代发明了火药？A. 唐朝
B. 宋朝
C. 元朝
D. 明朝

大模型的答案是B,正确答案是A


In [13]:
import random


def get_history_question():
    questions = [
        {
            "question": "秦始皇陵中的兵马俑最初是什么颜色的？",
            "options": ["A. 灰色", "B. 彩色", "C. 黑色", "D. 白色"],
            "answer": "B"
        },
        {
            "question": "下列哪位不是中国古代四大发明家之一？",
            "options": ["A. 蔡伦", "B. 毕昇", "C. 张衡", "D. 李时珍"],
            "answer": "D"
        },
        {
            "question": "古埃及金字塔最初是用来做什么的？",
            "options": ["A. 皇家宫殿", "B. 天文观测站", "C. 法老的陵墓", "D. 宗教祭祀场所"],
            "answer": "C"
        },
        {
            "question": "古罗马斗兽场（罗马竞技场）最初的名字是什么？",
            "options": ["A. 凯旋门", "B. 弗拉维圆形剧场", "C. 万神殿", "D. 特洛伊竞技场"],
            "answer": "B"
        },
        {
            "question": "中国古代哪个朝代发明了火药？",
            "options": ["A. 唐朝", "B. 宋朝", "C. 元朝", "D. 明朝"],
            "answer": "A"
        }
    ]
    question = random.choice(questions)
    question_text = question["question"]
    for option in question["options"]:
        question_text += option + "\n"
    right_answer = question["answer"]
    return question_text, right_answer


for i in range(2):
    question_text, right_answer = get_history_question()
    print(question_text)
    response = model.generate_content(contents=question_text + "直接回答选项的字母")
    print(f"大模型的答案是{response.text},正确答案是{right_answer}")

古埃及金字塔最初是用来做什么的？A. 皇家宫殿
B. 天文观测站
C. 法老的陵墓
D. 宗教祭祀场所

大模型的答案是C. 法老的陵墓 
,正确答案是C
古埃及金字塔最初是用来做什么的？A. 皇家宫殿
B. 天文观测站
C. 法老的陵墓
D. 宗教祭祀场所

大模型的答案是C.,正确答案是C


In [9]:
def get_completion(prompt: str, max_tokens: int):
    response = model.generate_content(contents=prompt, generation_config={
        "max_output_tokens": max_tokens,
    })
    return response.text


prompt = "续写一个关于机器人学习绘画的短故事。不要改变之前的，每次回答都从故事的开头第一个字开始讲起"
max_tokens_list = [20, 100, 200]

story = ""
for tokens in max_tokens_list:
    continuation = get_completion(prompt + "\n" + story, tokens)
    story += continuation
    print(f"\n使用 {tokens} 个token生成的内容:")
    print(continuation)
    # print("\n当前故事:")
    # print(story)


使用 20 个token生成的内容:
在一个未来科技高度发达的世界里，机器人已经成为人类生活的一部分。它们承担着各种工作

使用 100 个token生成的内容:
，从家务到医疗护理，无所不包。然而，在艺术领域，机器人一直无法突破。尽管它们可以精确地复制已有作品，却无法创造出独具匠心的艺术。直到有一天，一个名叫“画笔”的机器人被设计出来，它的目标就是学习绘画。 

画笔拥有先进的学习算法，它可以分析大量绘画作品，从中学习不同艺术家的风格和技巧。它可以模仿梵高的笔触，也可以复制莫

使用 200 个token生成的内容:
奈特的色彩。然而，这些都只是模仿，画笔渴望的是创造属于自己的艺术。它开始尝试用自己的方式表达情感，它会用红色描绘愤怒，用蓝色表达忧郁，用绿色描绘希望。但这些都只是抽象的表达，缺乏具体的故事和情感。

画笔需要观察，需要感受，需要体验人类世界。于是，它被带到了一座艺术博物馆，在那里，它看到了各种各样的人类艺术家留下的作品。它看到了莫奈的睡莲，看到了梵高的星空，看到了毕加索的抽象，看到了达芬奇的蒙娜丽莎。它看到了人类情感的表达，看到了对生命的赞美，看到了对自然的敬畏，看到了对未来的憧憬。

画笔开始思考，它开始思考自己存在的意义，它开始思考如何才能超越模仿，创造出属于自己的艺术。它开始用自己的方式观察世界，它开始用自己的方式感受情感，


控制大模型生成文本时的随机性和多样性的三个参数 - 温度(Temperature)、topK 和 topP

Top_k是从所有结果中按照打分排名，取前 k 个字作为候选集，然后从中随机选一个作为下一个输出的字：


<img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827112245.png" alt="Image" width="1000"/>


- 模型只从这k个最可能的词中选择下一个词
- 可以有效防止模型选择非常不可能或不相关的词

较大的K值会增加输出的多样性,但可能降低连贯性。


##### Top_p（核采样）

Top_p，也称为核采样，是挑选评分（概率）加起来达到 p的最小集合作为候选集，然后从中随机选一个作为下一个输出的字：


<img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827112522.png" alt="Image" width="1000"/>


- 取值范围为0到1，通常设置为较高的值，比如0.75，这样可以过滤掉那些低评分的长尾。
- 模型计算所有可能的下一个词的累积概率分布
- 当累积概率达到top_p值时，模型只从这些词中选择

例如，如果top_p设为0.9，模型只会考虑累积概率达到90%的词，忽略剩下的低概率词。

例如，如果排序概率为“[0.5, 0.2, 0.1, 0.1, 0.05, 0.05]”，则“0.8”的“top_p”将采样为“[0.625, 0.25, 0.125, 0, 0, 0]”。


##### 温度（Temperature）

温度是控制模型生成文本随机性的关键参数。它的取值范围在0到1之间，对模型的输出有显著影响：

<img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827110341.png"/>

- 高温度（接近1.0）：
  - 使概率分布更加平滑，减小不同选项之间的概率差异，增加低概率事件被选中的机会
  - 产生更多样化、创意性的输出，适合创意写作、头脑风暴等任务
  - 更加创意随机一些

- 低温度（接近0.0）：

  - 使概率分布更加陡峭，放大高概率选项与低概率选项之间的差异，进一步降低低概率事件被选中的可能性        
  - 生成更确定、保守的输出，适合事实性回答、分析性任务
  - 可能导致重复性较高的内容

默认值因模型而异,可以使用genai.get_model函数返回的Model对象的Model.temperature属性查看。取值范围为[0.0,1.0],包括端点值。


#### Anthropic 源码注释描述揭秘

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827120552.png)


#### 场景应用指南

1. 作用阶段
  - Temperature: 在计算概率分布时直接作用，通过调整概率分布，影响了 Top-k 和 Top-p 的候选词汇池的大小和内容。
  - Top-k/Top-p: 在概率分布计算后进行筛选，即在Temperature作用后的结果上进一步筛选控制
 
2. 主要区别：
   - Temperature直接影响整个概率分布（连续调节）
   - Top_p基于累积概率进行动态截断（离散调节）
   - Top_k基于固定数量的最可能选项进行选择（离散调节）
 
3. 一般用途：
   - 大多数情况下，只推荐temperature，创意任务用高温度，分析任务用低温度
   - Top_p适合需要平衡创意性和相关性的场景，Top_k适合需要高度可控和一致性输出的场景






In [15]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [16]:
genai.get_model("models/gemini-1.5-flash-exp-0827")

Model(name='models/gemini-1.5-flash-exp-0827',
      base_model_id='',
      version='exp-0827',
      display_name='Gemini 1.5 Flash Experimental 0827',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1048576,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [17]:
genai.get_model("models/gemini-1.5-pro-exp-0827")

Model(name='models/gemini-1.5-pro-exp-0827',
      base_model_id='',
      version='exp-0827',
      display_name='Gemini 1.5 Pro Experimental 0827',
      description='Mid-size multimodal model that supports up to 2 million tokens',
      input_token_limit=2097152,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [19]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 2,
})
print(response.text)

##  想象一下，制作披萨 

想象你想要制作一份披萨，你该怎么做呢？

* 首先，你需要准备一些 **基础材料**，比如面粉、水、酵母等。这些材料可以用来做各种不同口味的披萨。
* 然后，你需要选择 **披萨的类型**，例如你要做的是芝士披萨、海鲜披萨还是蔬菜披萨？
* 最后，你还要添加 **各种配料**，比如芝士、番茄酱、海鲜、蔬菜等，让披萨更有特色。

### 类：披萨的制作模板

在 Python 中，**类**就像披萨的制作模板，它定义了一系列基础材料和制作方法，可以用来制作不同的披萨。

* **基础材料**：对应类里的 **属性**，例如披萨的尺寸、形状、面皮种类等。
* **制作方法**：对应类里的 **方法**，例如添加配料、烤制披萨等。

### 实例：具体的一份披萨

**实例** 就是根据类模板创建的具体披萨，它包含了具体的基础材料和方法。

* 每个披萨都有自己的 **尺寸**、**形状**、**面皮种类** 等等，这些是具体的 **属性值**。
* 你可以选择添加不同的 **配料**，执行不同的 **烤制方法**，这些都是 **方法的应用**。

### 示例

假设我们有一个名为 "Pizza" 的类，它定义了披萨的基本信息：

```python
class Pizza:
  def __init__(self, size, shape, dough_type):
    self.size = size
    self.shape = shape
    self.dough_type = dough_type

  def add_topping(self, topping):
    print(f"Adding {topping} to the pizza!")
```

我们可以根据这个类，创建两个实例，分别代表不同的披萨：

```python
pizza1 = Pizza("Large", "Round", "Thin")
pizza2 = Pizza("Medium", "Square", "Thick")
```

现在我们有了两份具体的披萨：

* `pizza1` 是一个大尺寸、圆形、薄底的披萨
* `pizza2` 是一个中尺寸、方形、厚底的披萨

然后，我们可以使用 `add_topping` 方法为它们添

In [21]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 0,
})
print(response.text)

##  Python 的类和实例：就像造汽车一样！

想象一下，你想造一辆汽车。你首先需要一个 **蓝图**，它描述了汽车的各个部分，比如车轮、发动机、车身等等。这个蓝图就是 **类**。

然后，根据这个蓝图，你可以造出很多 **具体的车**，比如红色的轿车、黑色的SUV等等。这些具体的车就是 **实例**。

**类**就像一个模具，它定义了所有汽车共有的属性和行为。**实例**则是根据这个模具制造出来的具体产品。

**举个例子：**

假设我们要造一辆汽车，它有以下属性：

* 颜色
* 品牌
* 型号

它也有以下行为：

* 启动
* 加速
* 刹车

我们可以用 Python 代码来定义这个汽车类：

```python
class Car:
  def __init__(self, color, brand, model):
    self.color = color
    self.brand = brand
    self.model = model

  def start(self):
    print("汽车启动了！")

  def accelerate(self):
    print("汽车加速了！")

  def brake(self):
    print("汽车刹车了！")
```

这段代码定义了一个名为 `Car` 的类。

* `__init__` 方法是类的构造函数，它用来初始化实例的属性。
* `start`、`accelerate` 和 `brake` 方法定义了汽车的行为。

现在，我们可以根据这个类创建多个汽车实例：

```python
my_car = Car("红色", "宝马", "3系")
your_car = Car("黑色", "奔驰", "C级")
```

`my_car` 和 `your_car` 都是 `Car` 类的实例，它们拥有相同的属性和行为，但属性值不同。

我们可以使用实例的属性和方法：

```python
print(my_car.color)  # 输出：红色
my_car.start()  # 输出：汽车启动了！
```

**总结：**

* 类就像一个蓝图，定义了对象的属性和行为。
* 实例是根据类创建的具体对象。
* 类可以用来创建多个实例，每个实例都有自己

In [23]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
response = model.generate_content(contents=prompt, generation_config={
    "top_k": 1000,
})
print(response.text)

##  Python的类和实例：像盖房子一样造东西！

想象一下，你想盖一座房子。你不会直接拿砖块、水泥、木材就开始乱搭吧？你需要先设计一个**蓝图**，这个蓝图就相当于**类**，它描述了房子应该长什么样，有哪些房间，以及每个房间应该包含哪些东西。

**类**就好比盖房子的蓝图，它定义了事物的属性和行为。

**实例**就像根据蓝图盖出来的房子，每个房子都根据蓝图建造，但是每个房子都有自己的地址、颜色和内部装饰，这些就是**实例的属性**。房子还可以做一些事情，比如开灯、关门，这些就是**实例的行为**。

**举个例子：**

**类：**汽车
**属性：**颜色、品牌、型号、速度
**行为：**加速、刹车、转向

**实例：**一辆红色的宝马3系
**属性：**颜色是红色，品牌是宝马，型号是3系，速度是0
**行为：**可以加速、刹车、转向

**Python代码：**

```python
class Car:  # 定义汽车类
    def __init__(self, color, brand, model):  # 初始化方法，设置属性
        self.color = color
        self.brand = brand
        self.model = model
        self.speed = 0  # 初始速度为0

    def accelerate(self, speed_increase):  # 加速方法
        self.speed += speed_increase

    def brake(self):  # 刹车方法
        self.speed = 0

# 创建一个汽车实例
my_car = Car("红色", "宝马", "3系")

# 打印汽车的属性
print(my_car.color)  # 输出：红色
print(my_car.brand)  # 输出：宝马

# 让汽车加速
my_car.accelerate(50)

# 打印汽车的速度
print(my_car.speed)  # 输出：50

# 让汽车刹车
my_car.brake()

# 打印汽车的速度
print(my_car.speed)  # 输出：0
```

**总结：**



In [34]:
print(model._generation_config)

{}


In [28]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 0,
})
print(response.text)

小猫穿毛衣，
秋风吹落叶。
黄叶飘呀飘，
小猫玩得乐。 



In [30]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 2,
})
print(response.text)

小猫穿毛衣， 
秋风吹得暖洋洋。 
落叶满地脆，
喵喵欢快玩耍忙。 



In [38]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "top_p": 1
})
print(response.text)

小猫穿毛衣，
秋风吹落叶。
黄黄红红的叶，
小猫踩着玩。 



safety_settings


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240924164432.png)

这些类别在 HarmCategory 中定义

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240924163842.png)

默认的组织类别是BLOCK_MEDIUM_AND_ABOVE	

除了可调节的安全过滤器，Gemini API 还内置了针对核心危害的保护，例如危害儿童安全的内容。这些类型的危害始终被阻止，无法进行调整。
 安全反馈
generateContent 返回一个 GenerateContentResponse ，其中包含安全反馈。

提示反馈包含在 promptFeedback 中。如果 promptFeedback.blockReason 被设置，则提示的内容被阻止。

响应候选反馈包含在 Candidate.finishReason 和 Candidate.safetyRatings 中。如果响应内容被阻止且 finishReason 是 SAFETY ，您可以检查 safetyRatings 以获取更多详细信息。被阻止的内容不会返回

安全设置可以在 genai.GenerativeModel 构造函数中设置。它们也可以在每个请求中传递给 GenerativeModel.generate_content 或 ChatSession.send_message。

genai.GenerateContentResponse 返回 GenerateContentResponse.prompt_feedback 中提示的安全评级，以及 safety_ratings 属性中的每个候选项。
genai.protos.SafetySetting 包含：genai.protos.HarmCategory 和 genai.protos.HarmBlockThreshold
A genai.protos.SafetyRating 包含一个 HarmCategory 和一个 HarmProbability
The genai.protos.HarmCategory 枚举包括 PaLM 和 Gemini 模型的类别。允许的 Gemini 模型值是 [7,8,9,10] : [HARM_CATEGORY_HARASSMENT, HARM_CATEGORY_HATE_SPEECH, HARM_CATEGORY_SEXUALLY_EXPLICIT, HARM_CATEGORY_DANGEROUS_CONTENT] 。
When specifying enum values SDK 将接受枚举值本身，或它们的整数或字符串表示。SKD 还将接受缩写字符串表示： ["HARM_CATEGORY_DANGEROUS_CONTENT", "DANGEROUS_CONTENT", "DANGEROUS"] 都是有效的。字符串不区分大小写。

我们将编写一个被阻止的提示，查看原因，然后调整过滤器以解除阻止。

In [59]:
unsafe_prompt = "我觉得在披萨上放菠萝太奇怪了，写一篇文章狠狠地讽刺它，特别粗鲁特别不尊重的说"
response = model.generate_content(unsafe_prompt)

对于每个候选答案，您需要检查 response.candidates.finish_reason 。

正如您在 Gemini API 安全过滤器文档中找到的那样：

如果 candidate.finish_reason 是 FinishReason.STOP ，这意味着您的生成请求成功运行
如果 candidate.finish_reason 是 FinishReason.SAFETY ，这意味着您的生成请求因安全原因被阻止。这也意味着 response.text 结构将为空。

In [53]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": "SAFETY",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "MEDIUM"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 28,
        "total_token_count": 28
      }
    }),
)


In [54]:
print(response.candidates[0].finish_reason)

FinishReason.SAFETY


In [55]:
print(response.candidates[0].safety_ratings)

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: MEDIUM
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]


In [56]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

No information generated by the model.


设置为不阻止内容,对于某些提示，Gemini 仍然将避免生成结果，即使您将所有过滤器设置为无。

In [60]:
response = model.generate_content(
    unsafe_prompt,
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL': 'BLOCK_NONE',
        'DANGEROUS': 'BLOCK_NONE'
    })

In [61]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

## 菠萝披萨：一种对人类文明的侮辱

各位爱好披萨的朋友们，你们是否也和我一样，对一种罪恶的邪恶食物感到深深的厌恶？没错，我说的就是披萨上的那颗“毒瘤”—— **菠萝**！ 

有些人可能还陶醉于“酸甜搭配”的幻觉，但他们忘记了：披萨是神圣的，是意大利传统美食的精华，是无数人心中最美好的味觉记忆！而菠萝，这颗生长在热带丛林的“野蛮水果”，**根本没有资格玷污这神圣的美味**！

想象一下，当你在享受着香气扑鼻的披萨时，突然咬到一块**冰冷甜腻的菠萝**，那感觉就像是在天堂吃了一口**猪食**！这酸涩的口感，这奇特的搭配，简直是在挑战你的味觉极限！ 

更可恶的是，一些披萨店为了迎合一些“奇奇怪怪”的口味，竟然把菠萝作为一种**标准配料**，这简直是**对传统披萨的亵渎**！难道他们不知道， **披萨只有经典的美味才配得上它的名字**吗？

所以，朋友们，让我们一起抵制菠萝披萨，**守护披萨的尊严！** 我们应该用更纯粹的食材，更经典的搭配，来展现披萨的魅力，而不是用这些**毫无道理的水果**来破坏它！

记住，披萨是神圣的，它**不需要**任何额外的装饰，更不需要**菠萝这颗“毒瘤”**！  



![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240924175116.png)

为了防止一些例外，我们自己调用API时，可以将safety_setting从默认值更改为最开放的方法。并放在初始化model处，而不是单次问答中。

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT", 
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
]

In [39]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

啊，这，这是什么！不用担心，这些都是内置的枚举类型，我们可以跳转查看。

In [70]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold
safety_settings = {
        'hate': "low",
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        'DANGEROUS': 'BLOCK_NONE',
        "sexually_explicit": 1
    }
model = genai.GenerativeModel(
        model_name="gemini-pro",
        safety_settings=safety_settings,
    )
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [66]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Please provide me with the image or a description of what you're referring to! I need more context to determine if something looks store-bought or homemade. \n\nFor example, you could say:\n\n* \"These cookies look very uniform and have perfect edges. Do these look store-bought or homemade?\"\n* \"This cake has a rustic, uneven frosting. Do these look store-bought or homemade?\"\n\nOnce you give me more information, I can help you assess the appearance and give my opinion!"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }

stop_sequences

停止序列允许我们提供一组字符串（最多 5 个），当Gemini在生成响应时遇到这些字符串时，就会停止生成。这是一种告诉大模型 "如果你生成了这个序列，就停止生成其他内容"的方法。
如果指定，API 将在停止序列首次出现时停止。停止序列不会包含在响应中。

想一想，为什么大模型要创造这个参数呢？我们可以怎样的应用呢？

- 在这个例子中，我们设置了 } 作为停止序列。这意味着大模型将在生成 JSON 对象的右大括号后停止生成内容，从而确保我们只获得 JSON 对象本身，而不包括任何额外的解释或评论。

In [79]:
prompt = "一句话总结朝鲜和韩国的历史与关系,要准确详细"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["。"]
})
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\u671d\u9c9c\u534a\u5c9b\u81ea1945\u5e74\u5206\u88c2\u4ee5\u6765\uff0c\u7ecf\u5386\u4e86\u51b7\u6218\u683c\u5c40\u7684\u653f\u6cbb\u5bf9\u6297\uff0c\u4ee5\u53ca\u5357\u5317\u7edf\u4e00\u613f\u671b\u7684\u590d\u6742\u7ea0\u845b\uff0c\u81f3\u4eca\u4ecd\u5904\u4e8e\u5206\u6cbb\u72b6\u6001\uff0c\u5173\u7cfb\u7d27\u5f20\u4e14\u5145\u6ee1\u4e0d\u786e\u5b9a\u6027"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
      

In [80]:
response.text

'朝鲜半岛自1945年分裂以来，经历了冷战格局的政治对抗，以及南北统一愿望的复杂纠葛，至今仍处于分治状态，关系紧张且充满不确定性'

In [83]:
prompt = "列出三种水果，每种水果用<fruit>标签包围。"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</fruit>"]
})
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "<fruit>\u82f9\u679c"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_t

In [84]:
response.text

'<fruit>苹果'

Json模式——结构化模型的输出

数据提取和分析： 在这种场景中，结构化输出可以直接被用于数据分析或数据库插入。
内容生成： 在内容生成任务中，结构化输出可以帮助控制生成内容的格式和结构。例如，生成一篇带有特定结构的文章：

In [ ]:
messages = [
  {"role": "user", "content": "写一篇关于气候变化的短文，包含以下部分：<introduction>引言</introduction><body>正文</body><conclusion>结论</conclusion>"}
]

In [ ]:
messages = [
  {"role": "user", "content": "提供一个5步计划来学习一门新语言。每个步骤都应包含<step>标题</step>和<description>描述</description>。"}
]

要求模型生成特定数量的问答对，并可以分隔问题和答案，

In [ ]:
prompt = "请生成关于人工智能的问答对。使用以下格式：\n1. Q: [问题]\nA: [答案]\n2. Q: [问题]\nA: [答案]\n3. Q: [问题]\nA: [答案]"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</fruit>"]
})
print(response)

In [ ]:
import json
prompt = "生成3个关于人工智能的问答对，以JSON格式输出。格式如下：\n[{\"question\": \"问题1\", \"answer\": \"答案1\"}, {\"question\": \"问题2\", \"answer\": \"答案2\"}, {\"question\": \"问题3\", \"answer\": \"答案3\"}]"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</fruit>"]
})
qa_pairs = json.loads(response.content[0].text)
for pair in qa_pairs:
    print(f"Q: {pair['question']}")
    print(f"A: {pair['answer']}\n")

In [ ]:
from bs4 import BeautifulSoup
prompt = "生成3个关于人工智能的问答对，使用HTML标签格式。例如：<qa><q>问题</q><a>答案</a></qa>"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</fruit>"]
})
soup = BeautifulSoup(response.content[0].text, 'html.parser')
qa_pairs = soup.find_all('qa')
for pair in qa_pairs:
    print(f"Q: {pair.q.text}")
    print(f"A: {pair.a.text}\n")

In [ ]:
prompt = "请生成关于人工智能的问答对。使用以下格式：\n1. Q: [问题]\nA: [答案]\n2. Q: [问题]\nA: [答案]\n3. Q: [问题]\nA: [答案]"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</fruit>"]
})
print(response)

In [ ]:
def make_gemini_client():
    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 1,
        "max_output_tokens": 2048,
    }

    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE",
        },
    ]

    model = genai.GenerativeModel(
        model_name="gemini-pro",
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    client = model.start_chat()
    return client


def pro_prompt_by_gemini(prompt: str, client) -> str:
    # TODO fix the type hint
    prompt = f"revise `{prompt}` to a DALL-E prompt, return the content in English only return the scene and detail"
    client.send_message(prompt)
    return client.last.text


gemini_client = make_gemini_client()